In [19]:
import pandas as pd

# Load data
sentiment = pd.read_csv("/content/fear_greed_index.csv")
trades = pd.read_csv("/content/historical_data.csv")

#  Rename columns to standard names
sentiment.rename(columns={'date': 'Date'}, inplace=True)
trades.rename(columns={'Timestamp': 'time'}, inplace=True)

# Strip spaces (avoid hidden issues)
sentiment.columns = sentiment.columns.str.strip()
trades.columns = trades.columns.str.strip()

# Convert to datetime
sentiment['Date'] = pd.to_datetime(sentiment['Date']).dt.date
trades['time'] = pd.to_datetime(trades['time'])
trades['Date'] = trades['time'].dt.date

# Merge on Date column
merged = trades.merge(sentiment[['Date', 'classification']], on='Date', how='left')

print(merged.head())
print(merged.info())


                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

   Closed PnL                                   Transaction Hash     Order ID  \
0         0.0  0xec0945

In [20]:
import pandas as pd

# LOAD DATA
sentiment = pd.read_csv("/content/fear_greed_index.csv")
trades = pd.read_csv("/content/historical_data.csv")

#CLEAN & STANDARDIZE
# Rename date columns
sentiment.rename(columns={'date': 'Date'}, inplace=True)
trades.rename(columns={'Timestamp': 'time'}, inplace=True)

# Clean column names from trailing spaces
sentiment.columns = sentiment.columns.str.strip().str.lower()
trades.columns = trades.columns.str.strip().str.lower()

# Ensure date formats match
sentiment['date'] = pd.to_datetime(sentiment['date']).dt.date
trades['time'] = pd.to_datetime(trades['time'])
trades['date'] = trades['time'].dt.date

#  MERGE DATASETS
merged = trades.merge(
    sentiment[['date','classification']],
    on='date',
    how='left'
)

#  FEATURE ENGINEERING
# Win/Loss flag
merged['win_flag'] = (merged['closed pnl'] > 0).astype(int)

# Sentiment numeric mapping
merged['sentiment_binary'] = merged['classification'].map({'Fear':0, 'Greed':1})

# PnL % normalized (optional)
merged['pnl_pct'] = (merged['closed pnl'] / merged['size usd']) * 100

#  BASIC INSIGHTS
print("\n Average PnL by Market Sentiment")
print(merged.groupby("classification")['closed pnl'].mean())

print("\n Long vs Short Performance Under Fear/Greed")
print(merged.groupby(["classification","side"])['closed pnl'].mean())

print("\n Win Rate in Fear vs Greed")
print(merged.groupby("classification")['win_flag'].mean() * 100)

#  PREVIEW MERGED DATA
print("\n Final Merged Data")
print(merged.head())
print(merged.info())



 Average PnL by Market Sentiment
Series([], Name: closed pnl, dtype: float64)

 Long vs Short Performance Under Fear/Greed
Series([], Name: closed pnl, dtype: float64)

 Win Rate in Fear vs Greed
Series([], Name: win_flag, dtype: float64)

 Final Merged Data
                                      account  coin  execution price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   size tokens  size usd side     timestamp ist  start position direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   